# Spaceship Titanic
[Kaggle Competiton](https://www.kaggle.com/competitions/spaceship-titanic/)


    - PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
    - HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.
    - CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
    - Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
    - Destination - The planet the passenger will be debarking to.
    - Age - The age of the passenger.
    - VIP - Whether the passenger has paid for special VIP service during the voyage.
    - RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
    - Name - The first and last names of the passenger.
    - Transported - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.



In [4]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import xgboost as xgb
import optuna
from sklearn.metrics import accuracy_score


def convert(dataframe):
    dataframe["HomePlanet"]=dataframe["HomePlanet"].astype("category")
    dataframe["Destination"]=dataframe["Destination"].astype("category")
    dataframe[["group", "number_within_group"]]=dataframe["PassengerId"].str.split("_", expand=True)
    dataframe["group"]=dataframe["group"].astype(int)
    dataframe["number_within_group"]=dataframe["number_within_group"].astype(int)
    dataframe[["deck", "num", "side"]]=dataframe["Cabin"].str.split("/", expand=True)
    dataframe["deck"]=dataframe["deck"].astype("category")
    dataframe["num"]=dataframe["num"].astype("category")
    dataframe["side"]=dataframe["side"].astype("category")
    dataframe = dataframe.drop(["Cabin", "PassengerId", "Name"], axis=1) #also remove name
    dataframe["CryoSleep"] =dataframe["CryoSleep"].astype(bool)
    dataframe["VIP"] =dataframe["VIP"].astype(bool)
    dataframe["Age"]=pd.to_numeric(dataframe['Age'], errors='coerce', downcast='integer')
    dataframe["RoomService"]=pd.to_numeric(dataframe['RoomService'], errors='coerce', downcast='integer')
    dataframe["FoodCourt"]=pd.to_numeric(dataframe['FoodCourt'], errors='coerce', downcast='integer')
    dataframe["ShoppingMall"]=pd.to_numeric(dataframe['ShoppingMall'], errors='coerce', downcast='integer')
    dataframe["Spa"]=pd.to_numeric(dataframe['Spa'], errors='coerce', downcast='integer')
    dataframe["VRDeck"]=pd.to_numeric(dataframe['VRDeck'], errors='coerce', downcast='integer')
    return dataframe


mnist_train = pd.read_csv("/home/rainer/Downloads/ML_datasets/spaceship_titanic_train.csv")
mnist_train = convert(mnist_train)
mnist_train["Transported"] =mnist_train["Transported"].astype(bool)

X, Y = mnist_train.drop('Transported', axis=1), mnist_train[['Transported']]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)
dtrain_reg = xgb.DMatrix(X_train, Y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, Y_test, enable_categorical=True)

In [5]:
def objective(trial):
    param = {
        'tree_method':'gpu_hist',
        "objective": "binary:hinge",
        'eta': trial.suggest_float('eta', 0.01, 0.5),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.01, 10, log=True),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1),

    }

    boostround=100
    results = xgb.cv(
        params=param,
        dtrain=dtrain_reg,
        num_boost_round=boostround,
        nfold=5,
        verbose_eval=100, # print eval every xth round
        metrics=["logloss", "error"],
    )

    return results.iloc[-1]["test-error-mean"]

In [6]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=500)
print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

[I 2023-04-29 18:23:49,004] A new study created in memory with name: no-name-2b83ea95-8099-463a-995d-25a0cf4e5861


[0]	train-logloss:18.40655+0.12457	train-error:0.49962+0.00338	test-logloss:18.40644+0.49841	test-error:0.49961+0.01353
[99]	train-logloss:4.39818+0.11320	train-error:0.11938+0.00307	test-logloss:10.22331+0.35727	test-error:0.27750+0.00970


[I 2023-04-29 18:23:51,231] Trial 0 finished with value: 0.27749542113762954 and parameters: {'eta': 0.028757665278137445, 'max_depth': 4, 'min_child_weight': 0.013555839098992667, 'subsample': 0.613766713846053, 'colsample_bynode': 0.8789010228270929}. Best is trial 0 with value: 0.27749542113762954.


[0]	train-logloss:18.40655+0.12457	train-error:0.49962+0.00338	test-logloss:18.40644+0.49841	test-error:0.49961+0.01353
[99]	train-logloss:0.54395+0.06939	train-error:0.01476+0.00188	test-logloss:8.73717+0.55282	test-error:0.23716+0.01501


[I 2023-04-29 18:23:56,199] Trial 1 finished with value: 0.23715646761367112 and parameters: {'eta': 0.3978620946756406, 'max_depth': 10, 'min_child_weight': 3.363671203310271, 'subsample': 0.5911337167203177, 'colsample_bynode': 0.8105615186015924}. Best is trial 1 with value: 0.23715646761367112.


...

[0]	train-logloss:18.40655+0.12457	train-error:0.49962+0.00338	test-logloss:18.40644+0.49841	test-error:0.49961+0.01353
[99]	train-logloss:0.61459+0.05757	train-error:0.01668+0.00156	test-logloss:8.83880+0.25060	test-error:0.23992+0.00680


[I 2023-04-29 18:45:05,233] Trial 498 finished with value: 0.23991520276473827 and parameters: {'eta': 0.3164588361141489, 'max_depth': 8, 'min_child_weight': 1.2483858042751717, 'subsample': 0.947173952475632, 'colsample_bynode': 0.5278929446189343}. Best is trial 53 with value: 0.19849697959875512.


[0]	train-logloss:18.40655+0.12457	train-error:0.49962+0.00338	test-logloss:18.40644+0.49841	test-error:0.49961+0.01353
[99]	train-logloss:6.84524+0.18591	train-error:0.18580+0.00505	test-logloss:7.60676+0.10291	test-error:0.20647+0.00279


[I 2023-04-29 18:45:07,092] Trial 499 finished with value: 0.20647326368126412 and parameters: {'eta': 0.12042486518484762, 'max_depth': 5, 'min_child_weight': 7.4463115577466334, 'subsample': 0.2182309768928781, 'colsample_bynode': 0.18157222903626635}. Best is trial 53 with value: 0.19849697959875512.


Number of finished trials: 500
Best trial:
  Value: 0.19849697959875512
  Params: 
    eta: 0.13641333871191655
    max_depth: 7
    min_child_weight: 4.527383710014901
    subsample: 0.5401473217181034
    colsample_bynode: 0.1380431871118455


In [15]:
bestParams = study.best_trial.params
bestParams["tree_method"]= "gpu_hist"
bestParams["objective"]= "binary:hinge"
dfull = xgb.DMatrix(X, Y, enable_categorical=True)
model = xgb.train(
    params=bestParams,
    dtrain=dfull,
    num_boost_round=100
)
spaceship_test = pd.read_csv("/home/rainer/Downloads/ML_datasets/spaceship_titanic_test.csv")
spaceship_finalPred = convert(spaceship_test)
spaceship_finalPred_matrix = xgb.DMatrix(spaceship_finalPred, enable_categorical=True)
predictions = model.predict(spaceship_finalPred_matrix)
predictionsDf = pd.DataFrame(predictions, columns=["Transported"])
predictionsDf["PassengerId"]=spaceship_test["PassengerId"]
predictionsDf=predictionsDf[["PassengerId","Transported"]]
predictionsDf["Transported"]=predictionsDf["Transported"].astype(bool)
predictionsDf.to_csv('/home/rainer/Downloads/ML_datasets/spaceship_titanic_test_prediction.csv', index=False)

In [19]:
predictionsDf

In [ ]:
param = {
    'tree_method':'gpu_hist',
    "objective": "binary:hinge",
    'eta': trial.suggest_float('eta', 0.01, 0.5),
    'max_depth': trial.suggest_int('max_depth', 3, 10),
    'min_child_weight': trial.suggest_float('min_child_weight', 0.01, 10, log=True),
    'subsample': trial.suggest_float('subsample', 0.1, 1),
    'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1),

}